In [1]:
from keras.applications.vgg16 import VGG16
model = VGG16()
print(model.summary())

Using TensorFlow backend.


Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [2]:
import numpy as np
import time
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import os


In [3]:
path = r'/home/garycc227/code/transferL/training pics'

img_data_list = []
for dataset in os.listdir(path):
  img_list = os.listdir(path + '/' + dataset)
  print('Loading image from '+ dataset + '\n')
  for img in img_list:
    img_path = "{}/{}/{}".format(path, dataset,img)
    img = image.load_img(img_path, target_size=(224,224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0) #? why we expend one dim
    x = preprocess_input(x)
#     print('Input image shape', x.shape)
    img_data_list.append(x)


Loading image from horses

Loading image from dogs

Loading image from cats



In [5]:
img_data = np.array(img_data_list)
print(img_data.shape)
img_data = np.rollaxis(img_data,1,0)
print(img_data.shape)
img_data = img_data[0]
print(img_data.shape)

(606, 1, 224, 224, 3)
(1, 606, 224, 224, 3)
(606, 224, 224, 3)


In [6]:
num_classes = 3
num_of_samples = img_data.shape[0]
labels = np.ones((num_of_samples,), dtype='int64')

names = ['horse', 'dog','cat']
labels[0:202] = 0
labels[202:404]= 1
labels[404:606]=2

Y = np_utils.to_categorical(labels, num_classes)

x,y = shuffle(img_data,Y, random_state=2)
X_train, X_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state = 2)

image_input = Input(shape = (224,224, 3))
model = VGG16(input_tensor = image_input, weights = 'imagenet')
model.summary()
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input,out)
custom_vgg_model.summary()



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [16]:
for layer in custom_vgg_model.layers[:-1]:
  layer.trainable = False

print(custom_vgg_model.layers[1].trainable)

False


In [22]:
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

t = time.time()
hist = custom_vgg_model.fit(X_train,y_train, batch_size = 16 , epochs=1, verbose=1, validation_data=(X_test, y_test))
# print('Training time: %s' %(t-time.time()))
# (loss, accuracy) = custom_vgg_model.evaluate(X_test, y_test, batch_size = 10, verbose=1)

# print('loss={:.4f}, accuracy: {:.4f}%'.format(loss, accuracy * 100))

Train on 484 samples, validate on 122 samples
Epoch 1/1


ResourceExhaustedError:  OOM when allocating tensor with shape[16,224,224,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node block1_conv1_1/Relu (defined at /home/garycc227/.venv/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3009) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_keras_scratch_graph_3618]

Function call stack:
keras_scratch_graph
